In [ ]:
import pandas as pd
import utils, config
from ml import preprocess, train_expected_returns
from importlib import reload

reload(preprocess)

In [ ]:
files = utils.list_s3_files(prefix="clean/", bucket_name=config.BUCKET_NAME)
f = files[0]
df = utils.read_s3_file(f)
df = preprocess.prepare_training_data(df)

In [ ]:
model_dict = train_expected_returns.train_expected_returns(df)

In [ ]:
df.head()

In [ ]:
preprocess.calculate_momentum(df).reset_index().set_index("date")

In [ ]:
def calculate_momentum(df, periods=[3, 6, 12]):
    df = df.copy()
    for period in periods:
        df = df.reset_index()
        momentum_feature = (
            df.groupby("ticker")["return_1m"]
            .rolling(window=period)
            .apply(lambda x: (x + 1).prod() - 1)
        )
        df = df.set_index(["date", "ticker"])
        df[f"momentum_{period}m"] = momentum_feature.values

    return df

In [ ]:
raw = calculate_momentum(raw)

In [ ]:
raw

In [ ]:
df = (
    utils.read_s3_file("output/er_forecasts_momentum.csv")
    .set_index("date")
    .drop(columns=["portfolio"])
)

In [ ]:
(df.drop(columns=["true_value"]).sub(df["true_value"], axis=0) ** 2).mean()